Just importing stuff

In [ ]:
%pip install keras
import pandas as pd
import numpy as np
import tensorflow as tf



In [ ]:
%pip install PyQt5
import PyQt5
import matplotlib.pyplot as plt



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.optimizers import Adam

In [ ]:
data = pd.read_csv("../complete_merged.csv",index_col=0)

Consecutive date checking function (Some districts had partial data)

In [ ]:
#Testing datetime consecutiveness
import datetime as dt

def checkConsecutive(date_strs):
    dates = [dt.datetime.strptime(d, "%Y-%m-%d") for d in date_strs]

    date_ints = set([d.toordinal() for d in dates])
    if len(date_ints) == 1:
        return True
    elif max(date_ints) - min(date_ints) == len(date_ints) - 1:
        return True
    else:
        return False




# date_strs = ['02-29-2012', '02-28-2012', '03-01-2012']
date_strs = ['1999-12-31','2000-01-01']
checkConsecutive(date_strs)



Split Sequence(Standard function for splitting data for univariate lstm)

In [ ]:


def split_sequence(date_sequence,temp_sequence, n_steps):
    X, y = list(), list()
    for i in range(len(temp_sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
 # check if we are beyond the sequence
        if end_ix > len(temp_sequence)-1:
            break
 # gather input and output parts of the pattern
        seq_x, seq_y = temp_sequence[i:end_ix], temp_sequence[end_ix]
        date_seq = date_sequence[i:end_ix]
        # print(checkConsecutive(seq_x))
        if(checkConsecutive(date_seq)):
            X.append(seq_x)
            y.append(seq_y)
        
    return np.array(X), np.array(y)





In [ ]:
#Basic Check
X,y = split_sequence(['2021-01-09','2021-01-11','2021-01-12','2021-01-13','2021-01-14'],[10, 20, 30, 40, 50],3)
for i in range(len(X)):
    print(X[i], y[i])

In [ ]:
def convertUnivariateData(date_data,temp_data,n_steps,n_features):
    X, y = split_sequence(date_data,temp_data, n_steps)
# summarize the data
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    return (X,y)

Defining the model

In [ ]:
#different optimizers and loss functions need to be tested yet
n_steps = 10
n_features = 1
epochs =20
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')



Function to evaluate model(Mean Square Error)

In [ ]:
def evalLSTM(model,X,y,n_steps,n_features):
    y_predict = list()
    for i in range(X.shape[0]):
        x_input = X[i]
        x_input = x_input.reshape((1, n_steps, n_features))
        predicted_value = model.predict(x_input, verbose=0)[0][0]
        print(len(y_predict),"of",X.shape[0])
        y_predict.append(predicted_value)
    
    mse = tf.keras.losses.MeanSquaredError()
    return {'mse':mse(y_predict,y).numpy(),'y_predict':y_predict}


Basic check for Adilabad

In [ ]:
data_Adilabad = data[data['district'] == "Adilabad"].sort_values(axis = 0,by = "date")

In [ ]:
data_Adilabad_temp = data_Adilabad['temp_max'].to_list()
data_Adilabad_date = data_Adilabad['date'].to_list()
# print(data_Adilabad_date)
X,y = convertUnivariateData(data_Adilabad_date,data_Adilabad_temp,n_steps,n_features)
model.fit(X, y, epochs=200, verbose=4)
mse = evalLSTM(model,X,y,10,1)
mse
#3.36 for 200 epochs
%matplotlib qt


plt.plot(y)
plt.plot(mse['y_predict'])
plt.show()


Various window sizes were used on Adilabad for optimization.Results were as follows:

[10-3.289,
 20-55142.400,
 30-6766769.447,
 40-216242835.855,
 50-14841958357.522,
 60-793333303283.484]


[7-116.768,
 8-28.928,
 9-5.358,
 10-3.289,
 11-5.317]

Concluding that 10 is an optimal window size

In [ ]:
def get_data(district,column):
    return data[data['district'] == district].sort_values(axis = 0,by = "date")[column].to_list()
   

Training on complete data of all districts

In [ ]:
i = 1
for district in data['district'].unique():
    print(str(i) + "out of "+str(len(data['district'].unique())))
    
    data_unique_district_temp = get_data(district,'temp_max')
    data_unique_district_date = get_data(district,'date')
    
    print(district,str(len(data_unique_district_temp)))
    X,y = convertUnivariateData(data_unique_district_date, data_unique_district_temp,n_steps,n_features)
    model.fit(X, y, epochs=epochs, verbose=0)
    i = i+1
    
    # print(data_unique_district_temp)

#mse = 3.042 for epochs = 20 for Hanumakonda
#mse = 4.34 for Adilabad

'mse': 3.042855618505585 when trained on all data and tested against random 20 epochs

In [ ]:
data_unique_district_temp = get_data('Adilabad','temp_max')
data_unique_district_date = get_data('Adilabad','date')
    
  
X,y = convertUnivariateData(data_unique_district_date, data_unique_district_temp,n_steps,n_features)
mse = evalLSTM(model,X,y,10,1)
plt.plot(y)
plt.plot(mse['y_predict'])
plt.show()